In [1]:
# 최신순으로 최근 3달 리뷰 크롤링
# 날짜, 리뷰 내용
# 크롬 108.xx 버전

In [2]:
import pandas as pd
import requests
from bs4 import BeautifulSoup

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import NoSuchElementException

import time
from datetime import datetime

In [4]:
def findCafe(phone, name):
    url = f'https://m.map.naver.com/search2/searchMore.naver?query={phone}&page=1'
    response = requests.get(url)
    places = response.json()['result']['site']['list']
    for place in places:
        if place['name'] == name:
            return place['id'][1:]

In [8]:
def dateCal(str):
    date = str.split(" ")
    year = int(date[0][:-1])
    month = int(date[1][:-1])
    day = int(date[2][:-1])
    
    return datetime(year, month, day)

In [3]:
cafeName = '팡팡팡'
cafeNum = '053-252-2025'
cafeID = ""

In [5]:
cafeID = findCafe(cafeNum, cafeName)
cafeID

'1347974588'

In [6]:
finalurl = f'https://m.place.naver.com/restaurant/{cafeID}/review/visitor?reviewSort=recent'

In [7]:
driver = webdriver.Chrome()
driver.get(finalurl)

In [9]:
now = datetime.now()

while True: 
        try: 
            driver.find_element(By.TAG_NAME, 'body').send_keys(Keys.END)
            time.sleep(1)
            driver.find_element(By.TAG_NAME, 'body').send_keys(Keys.PAGE_UP)
            time.sleep(1) 
            
            date = driver.find_element(By.CSS_SELECTOR, '.eCPGL > li:nth-last-child(1) > div:nth-last-child(1) > span > span:nth-last-child(1)').text
            
            dt = dateCal(date)
            diff = (now-dt).total_seconds()//86400
            
            if diff > 90: # 맨 마지막 리뷰 일자가 90일 이전이면 멈춤
                break
            
            driver.find_element(By.CSS_SELECTOR, '.lfH3O > a').click()
            time.sleep(2)           
            
            
        except NoSuchElementException: 
            print('-더보기 버튼 모두 클릭 완료-') 
            break 

4.0
9.0
12.0
18.0
20.0
24.0
27.0
29.0
32.0


KeyboardInterrupt: 

In [10]:
html = driver.page_source 
dom = BeautifulSoup(html, 'lxml')

In [21]:
reviews = dom.select('.YeINN > .ZZ4OK > a > span')
dates = dom.select('.YeINN > div:nth-last-child(1) > span > span:nth-last-child(1)')
len(reviews)

96

In [28]:
cafes = []
days = []
re = []

In [29]:
try: 
    for date, review in zip(dates, reviews): 
        
        day = dateCal(date.text)
        text = review.text
    
        cafes.append(cafeName)
        days.append(day)
        re.append(text)

# 리뷰가 없는 경우        
except NoSuchElementException: 
    print("네이버 리뷰 없음" )

In [31]:
df = pd.DataFrame({'cafeName':cafes,
                   'date': days,
                   'review': re
                   })

In [32]:
df.head()

,cafeName,date,review
0,팡팡팡,2022-12-06,조용하고넓어서좋아요
1,팡팡팡,2022-12-05,10시50분 마감이라 늦게까지 하고 넓어서 좋아요 하지만 10시이후는 드립이나 디카...
2,팡팡팡,2022-12-04,빵이 조금 달아요 커피는 맛있어요
3,팡팡팡,2022-12-04,매장두 넓고 빵들이 미쳤어요 ㅠㅠ\n비슷한 곳으로 디저트3*가 있는데 거긴 너무 어...
4,팡팡팡,2022-12-04,대구 최고의 베이커리 카페입니다!
